In [12]:
import pandas as pd

# 读取 Excel 文件
df = pd.read_excel('/Users/zhaqian/Desktop/DATA SCIENCE/CS老师教课资源/python爬虫/Issue_Management_Database.xlsx',sheet_name='issue management test(2)')  # 确保你的文件路径正确

In [13]:
print(df.head())

   Incident Information - Incident ID  \
0                                3667   
1                                3675   
2                                3676   

  Incident Information - Subject (Incident URL)  \
0                            JSS - 01312024 - 1   
1                            JSS - 01312024 - 2   
2                            JSS - 01312024 - 3   

                                         Issue Title    EVP Sponson  \
0  1. CAMELS: Operational Risk - Internal Control...   Mark DeFazio   
1  2. Operational Risk â€“ Internal Controls â€“ ...  Gregory Gaare   
2  MBHC 1. Operational Risk - Internal Controls â...     Erjun Chen   

             Business Area(s)       Issue Owner Issue Owner's email address  \
0              Internal Audit          Qian Zha      qian.zha32@outlook.com   
1  Enterprise Risk Management          Zhenhuan        xzhrobert@gmail.com    
2              Internal Audit  \nJake Nicholson      qian.zha32@outlook.com   

  Original Issue Date Issue 

In [14]:
from datetime import datetime, timedelta

In [15]:
# 假设有一列叫 'Target Date' 和 'Owner Emails'
df['Target Closure Date'] = pd.to_datetime(df['Target Closure Date'])

In [16]:
# 获取当前日期
current_date = datetime.now()

# 过滤出目标日期大于当前日期的记录
filtered_df = df[df['Target Closure Date'] > current_date]

# 计算剩余的天数
filtered_df['Days Left'] = (filtered_df['Target Closure Date'] - current_date).dt.days


In [17]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Gmail SMTP 配置
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
SENDER_EMAIL = 'qian.zha32@gmail.com'  # 替换为你的 Gmail 地址
SENDER_PASSWORD = 'rpvv lqid txxp oauy'  # Gmail 的应用专用密码

# 示例乱码字符映射（可以根据实际情况调整）
char_map = {
    'â€“': '-',  # 替换破折号
    'â€™': "'",  # 替换单引号
    'â€œ': '"',  # 替换左双引号
    'â€': '"',   # 替换右双引号
    'â€¦': '...',  # 替换省略号
    'Ã©': 'é',   # 拉丁字符 'é'
    'Ã': 'A',    # 拉丁字符 'A'
}

# 自动清理乱码字符的函数
def clean_text(text):
    if isinstance(text, str):  # 只对字符串类型的内容进行处理
        for key, value in char_map.items():
            text = text.replace(key, value)
    return text

def send_email(to_email, subject, body):
    msg = MIMEMultipart()
    msg['From'] = SENDER_EMAIL
    msg['To'] = to_email
    msg['Subject'] = subject

    # 添加邮件正文
    msg.attach(MIMEText(body, 'plain'))

    # 连接到 Gmail SMTP 服务器并发送邮件
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()  # 启用 TLS 加密
        server.login(SENDER_EMAIL, SENDER_PASSWORD)  # 使用应用专用密码登录
        server.sendmail(SENDER_EMAIL, to_email, msg.as_string())  # 发送邮件
        print(f"邮件已成功发送至 {to_email}")



邮件已成功发送至 qian.zha32@outlook.com


In [18]:
# 遍历过滤后的数据
for index, row in filtered_df.iterrows():
    # 分割多个邮箱地址（假设邮箱是以逗号分隔的）
    email_list = row['Issue Owner\'s email address'].split(',')
    
    # 邮件内容
    subject = f"Issue Reminder: {row['Issue Title']}"
    body = f"Dear {row['Issue Owner']},\n\nThis is a reminder that the target date for issue '{row['Issue Title']}' is approaching.\nYou have {row['Days Left']} days left.\n\nPlease take necessary action.\n\nBest regards,\nYour Team"

    # 发送邮件给每个邮箱
    for email in email_list:
        send_email(email.strip(), subject, body)


邮件已成功发送至 qian.zha32@outlook.com
邮件已成功发送至 xzhrobert@gmail.com
邮件已成功发送至 qian.zha32@outlook.com
